<a href="https://colab.research.google.com/github/Vl-Leschinskii/jokes_topics/blob/main/functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Функции использованные в статье, которые написаны самостоятельно, но в основном это готовые решения с просторов...

### Получение эмбеддингов из предобученной модели 

In [ ]:
def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=False, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

### Функция косинусной близости

In [ ]:
def similarity(v1, v2):
    n1 = np.linalg.norm(v1)
    n2 = np.linalg.norm(v2)

    if n1 < 1e-6 or n2 < 1e-6:
        return 0.0
    else:
        return np.dot(v1, v2) / n1 / n2

def word_similarity(model, w1, w2):
    return similarity(model.get_word_vector(w1), model.get_word_vector(w2))

def sentence_similarity(model, t1, t2):
    return similarity(model.get_sentence_vector(t1), model.get_sentence_vector(t2))

### Функции визуализации топиков

In [ ]:
def display_topics_gensim(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
      print("Topic %d:" % (topic_idx))
      print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [ ]:
def display_topics_sklearn(model, feature_names, no_top_words):
    topic_list=[]
    no_top_words=1000  
    for topic_idx, topics in enumerate(model.components_):
      sort_topix=topics.argsort()[:-no_top_words - 1:-1]
      to_idx=[topic_idx]    
      top_list=[]
      print("Topic %d:" % (topic_idx))
      line=[]       
      for i in sort_topix:
        top_list=[]       
        top_list.append(topic_idx)
        top_list.append(feature_names[i])
        top_list.append(topics[i])
        topic_list.append(top_list)
        line.append(feature_names[i])        
      print(line[:10])           
    return pd.DataFrame(topic_list,columns=['topic','topic_word','topic_val'])#,topic_list_2

### Строит визуализацию облака слов топиков для модели **Gensim**.

In [ ]:
def plotWordCloud_Gensim(topic_number,topics,head):
    text = dict(topics[topic_number][1])
    wordcloud = WordCloud(background_color="white", max_words=100, width=900, height=900, collocations=False)
    wordcloud = wordcloud.generate_from_frequencies(text)
    plt.figure(figsize=(15, 10))
    plt.title((head+" номер {}").format(topic_number))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off");

### Строит визуализацию облака слов топиков для модели **Sklearn**.

In [ ]:
# облако слов топиков для модели sklearn
def plotWordCloud_Sklearn(topic_number,topics):
    # получаем частоты и слова топика
    topic=topic_number
    words=topics[topics['topic']==topic]['topic_word'].values.astype('U')
    long_string =','.join(words)
    # строим облако слов
    wordcloud = WordCloud(background_color="white", max_words=100, width=900, height=900, collocations=False)
    wordcloud.generate(long_string)
    plt.figure(figsize=(15, 10))
    plt.title("Топик номер {}".format(topic_number))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off");

### Natural Language Toolkit

In [ ]:
nltk_download("punkt")
!wget https://raw.githubusercontent.com/dhhse/dh2020/master/data/stop_ru.txt
with open ("stop_ru.txt", "r") as stop_ru:
    rus_stops = [word.strip() for word in stop_ru.readlines()]
punctuation = '!\"#$%&\'()*+,-./:;<=>?@[\]^_`{|}~—»«...–'    
filter_token = rus_stops + list (punctuation)
from pymorphy2 import MorphAnalyzer
parser = MorphAnalyzer()

### Функции для предобработки текста. 

Слова приводятся к нижнему регистру,
стоп-слова удаляются, далее слова лемматизируются


input_text - Входной текст для очистки и лемматизации

return
  out_text - Очищенный и лемматизированный текст, как последовательность разделенных пробелами токенов
  lemmatized_text - Очищенный и лемматизированный текст, как список токенов

In [ ]:
def preprocess_sklearn(input_text):
    text = input_text.lower()
    tokenized_text = word_tokenize(text)
    clean_text = [word for word in tokenized_text if word not in filter_token]
    lemmatized_text = [parser.parse(word)[0].normal_form for word in clean_text]
    out_text = " ".join(lemmatized_text)    
    return out_text

def preprocess_gemsim(input_text):
    text = input_text.lower()
    tokenized_text = word_tokenize(text)
    clean_text = [word for word in tokenized_text if word not in filter_token]
    lemmatized_text = [parser.parse(word)[0].normal_form for word in clean_text]    
    return lemmatized_text

### Функция назначает документу наиболее вероятный топик

*words* -  лемматизированный текст документа

*lda* -  тематическая модель

*return* -  список из наиболее вероятного топика и его вероятности

In [ ]:
def get_topic(words, lda):
    bag = lda.id2word.doc2bow(words)
    topics = lda.get_document_topics(bag)
    topic_dictionary = {}
    for topic in topics:
        topic_dictionary[topic[1]] = str((topic[0])) 
    main_probability = max(topic_dictionary)
    main_topic = topic_dictionary[main_probability]
    return [main_topic, main_probability]

### Получение lda модели в библиотеке **Gensim**

In [ ]:
def lda_model_gensim(corpus,dictionary,num_topics):
  ldamodel = models.ldamodel.LdaModel(
    corpus, 
    id2word=dictionary, 
    eval_every=20, 
    num_topics=num_topics, 
    passes=5
    )
  return ldamodel

### Функции для сохранения топиков в файл

In [ ]:
def topic_file(topics_gensim_np,file_name):
  list_topic=[]
  for ind_topiс,topiс in enumerate(topics_gensim_np):
    list_word=topiс[1]
    for words in list_word:
      topic_word=[]
      topic_word.append(ind_topiс)
      topic_word.append(words[0])
      topic_word.append(words[1])
      list_topic.append(topic_word)
  topic_gensim=pd.DataFrame(list_topic,columns=['topic','word_in topic','weght_topic'])
  topic_gensim.to_csv(file_name+'.csv',index=False)
  return topic_gensim

In [ ]:
def topics_max(mod_data,model):
  top_n=[]
  for i in range(mod_data.shape[0]):
#for i in range(100):
    top_max=[]
    topic_list=model.transform(tfidf[i,:])
    top_max.append(np.max(topic_list))
    top_max.append(np.argmax(topic_list))  
    top_n.append(top_max)
  return np.array(top_n)

### Генератор анекдотов

In [ ]:
def joke_generator(data,topic_id,num,topic_model='topic_lda',):
  if topic_model=='topic_base': 
    data_topic=topic_base
  if topic_model=='topic_bi': 
    data_topic=topic_bi
  if topic_model=='topic_nmf': 
    data_topic=topic_nmf
  if topic_model=='topic_lda': 
    data_topic=topic_lda
  word_in_topic=data_topic[data_topic['topic']==topic_id]['topic_word'].values[:10]
  print('токены топика:',", ".join(word_in_topic) )
  print()

#  plotWordCloud_2(topic_number=topic_id, topics=data_topic)
  jokes=data[data[topic_model]==topic_id]['text_base'].values  
  for i, joke in enumerate(jokes[:num]):
    print('анекдот',i,joke)

### Построение биграмм

In [ ]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]